# Running Contour-Integration Model Against Adversarial Attacks

In [1]:
from __future__ import division, print_function, absolute_import
import pickle
import numpy as np

import keras.backend as K
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.imagenet_utils import decode_predictions, preprocess_input

from networks.CI_alex_net import alex_net
from attack import PixelAttacker

Using TensorFlow backend.


In [2]:
# Make a dictionary that maps the name of the class (in like real english) to the index
with open('data/imagenet_classes.pkl', 'rb') as f:
    class_names = pickle.load(f)
word_to_class = {w:i for i,w in enumerate(class_names)}

In [26]:
# Load images

# This probably needs to be made more efficient
# The idea is that we need a labels array that matches the image array
fnames = ('data/ILSVRC2012_val_00001036.JPEG', 'data/ILSVRC2012_val_00000655.JPEG', 
          'data/ILSVRC2012_val_00009401.JPEG', 'data/ILSVRC2012_val_00000962.JPEG', 
          'data/ILSVRC2012_val_00000911.JPEG', 'data/ILSVRC2012_val_00000329.JPEG', 
          'data/ILSVRC2012_val_00002138.JPEG', 'data/ILSVRC2012_val_00000293.JPEG')
names = ('wallaby, brush kangaroo', 'wallaby, brush kangaroo', 
         'water ouzel, dipper', 'water ouzel, dipper', 
         'cock', 'cock', 
         'tench, Tinca tinca', 'tench, Tinca tinca')

images = []
labels = []
for fname,name in zip(fnames,names):
    img = load_img(fname, target_size=(227, 227))
    x = img_to_array(img)
    images.append(x)
    labels.append([word_to_class[name]])
    
images = np.array(images)
labels = np.array(labels)

In [3]:
K.set_image_dim_ordering('th')
model = alex_net('networks/models/alexnet_weights.h5')

In [24]:
prediction = model.predict(images[0][np.newaxis, ...])

In [25]:
decode_predictions(prediction)

[[(u'n02342885', u'hamster', 0.001858573),
  (u'n03483316', u'hand_blower', 0.0018076637),
  (u'n04399382', u'teddy', 0.0016949538),
  (u'n02364673', u'guinea_pig', 0.0016129097),
  (u'n02174001', u'rhinoceros_beetle', 0.0015867078)]]

In [27]:
# we could evaluate more than one model
models = [model]

# The data set we will try to attack
test = images, labels

attacker = PixelAttacker(models, test, class_names, dimensions=(227,227))

Evaluating model_1


In [29]:
result = attacker.attack(6, model, maxiter=20, verbose=True)

Confidence: 0.0006411272


In [12]:
untargeted = attacker.attack_all([model], info='_Salman-CI_', samples=100, pixels=(1,3,5), targeted=False, maxiter=75, popsize=400, verbose=False)

TypeError: attack_all() got an unexpected keyword argument 'info'

In [12]:
targeted = attacker.attack_all([model], info='_Salman-CI_', samples=100, pixels=(1,3,5), targeted=True, maxiter=75, popsize=400, verbose=False)

TypeError: attack_all() got an unexpected keyword argument 'info'

In [ ]:
columns = ['model', 'pixels', 'image', 'true', 'predicted', 'success', 'cdiff', 'prior_probs', 'predicted_probs', 'perturbation']
untargeted_results = pd.DataFrame(untargeted, columns=columns)
targeted_results = pd.DataFrame(targeted, columns=columns)

In [ ]:
# untargeted results
helper.attack_stats(untargeted_results, models, network_stats)

In [ ]:
# targeted results
helper.attack_stats(targeted_results, models, network_stats)